# Horton-Rogers-Lapwood problem (natural convection in a hot room)

In [ ]:
from dolfin import *
from fe
def gravity(u):
    '''
    This function defines the right-hand-side source term in Eq. 7,
    and Ra is set to be 1.
    '''
    Ra = 1.0
    val = as_vector([0.0, Ra*u])
    return val
def solver(nx, order, t, dt, T):
    '''
    This function is to solve the manufactured solution problem
    described in Section 5.1.
    nx: number of vertices when creating a unit square mesh
    order: approximation order of pressure field
    t: simulation start time
    dt: fixed time step
    T: simulation stop time
    '''
    # Create a unite square mesh
    mesh = UnitSquareMesh(nx, nx)
    # Define function spaces
    # Pressure
    DG0 = FunctionSpace(mesh, "DG", order)
    # Velocity
    BDM = FunctionSpace(mesh, "BDM", order+1)
    # Temperature
    DG1 = FunctionSpace(mesh, "DG", order+1)
    W = MixedFunctionSpace([DG0, BDM, DG1])
    # f_1, f_2, f_3 are source terms in the manufactured problem
    f_1 = Expression('(sin(x[0]) + cos(x[1]))*cos(t)', t=t)
    f_2 = Expression(('0.0', '(-sin(x[0]) - sin(x[1]))*cos(t)'),
                     t=t)
    f_3 = Expression(-(sin(x[0]) + sin(x[1]))*sin(t) +
                     (-cos(x[0])*cos(x[0]) + sin(x[1])*cos(x[1]))*cos(t)*cos(t)
                     + (sin(x[0]) + sin(x[1]))*cos(t), t=t)
w = Function(W)
w0 = Function(W)
 # Initialize function according to exact solution expression
assign(w.sub(0), interpolate(p_exact, DG0))
assign(w.sub(1), interpolate(uu_exact, BDM))
assign(w.sub(2), interpolate(u_exact, DG1))
assign(w0.sub(0), interpolate(p_exact, DG0))
assign(w0.sub(1), interpolate(uu_exact, BDM))
assign(w0.sub(2), interpolate(u_exact, DG1))
# split w into p (pressure), uu (velocity) and u (temperature)
(p, uu, u) = split(w)
(p0, uu0, u0) = split(w0)
(q, vv, v) = TestFunctions(W)
n = FacetNormal(mesh)
# Penalty weights
alpha = Constant(5000.0)
gamma = Constant(10000.0)
# cell size
h = CellDiameter(mesh)
# Eq. 20
F_flo = nabla_div(uu)*q*dx - f_1*q*dx
# Eq. 21
F_vel = (dot(uu, vv) - div(vv)*p - inner(gravity(u), vv) -
          inner(f_2, vv) )*dx + dot(n, vv)*p_exact*ds
# un = un on outflow facet, otherwise 0
un = (dot(uu, n) + abs(dot(uu, n)))/2.0
# Eq. 26 (Note that the manufactured uu is not divergence-free.)
a_a = dot(grad(v), -uu*u)*dx - u*v*nabla_div(uu)*dx - f_3*v*dx
a_a += dot(jump(v), un('+')*u('+') - un('-')*u('-') )*dS
a_a += dot(v, un*u)*ds
# Eq. 28
a_d = dot(grad(v), grad(u))*dx\
 - dot(avg(grad(v)), jump(u, n))*dS - dot(jump(v, n),
 avg(grad(u)))*dS\
 - dot(grad(v), n*u)*ds - dot(v*n, grad(u))*ds\
 + (alpha('+')/h('+'))*dot(jump(v, n), jump(u, n))*dS +
 (gamma/h)*v*u*ds\
 + u_exact*dot(grad(v), n)*ds - (gamma/h)*u_exact*v*ds
a_tim = (u - u0)/dt*v*dx
# Eq. 29
F_a_d = a_tim + a_a + a_d
# Eq. 32
F = F_flo + F_vel + F_a_d
# Time-stepping loop
while t < T:
    t += dt
    print "nx=%d order=%d t=%f" % (nx, order, t)
    # Update time in time-dependent expressions
    f_1.t = t; f_2.t = t; f_3.t = t
    u_exact.t = t; p_exact.t = t; uu_exact.t = t
# Solve the system and store solution in w
    solve(F==0, w)
    # Update w0 with w
    w0.vector()[:] = w.vector()
    # Save solution in .xml file
    File("solution.xml") << w


SyntaxError: invalid syntax (3314692168.py, line 76)